# Setup

In [1]:
from consav import runtools
runtools.write_numba_config(disable=0,threads=4)

In [2]:
%matplotlib inline
# reload module each time cell is run
%load_ext autoreload
%autoreload 2
from ModelSingles import SingleClass
#from ModelCouples import CoupleClass
import numpy as np
import matplotlib.pyplot as plt
import funs
import figs

In [3]:
# Test that numba is working
# It should throw an error
from numba import njit, typeof
@njit(parallel=True)
def test():
    return [1, [2]]
test()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mType of variable '$0.4' cannot be determined, operation: build_list(items=[Var($const0.1, <ipython-input-3-ff937188aec6> (6)), Var($0.3, <ipython-input-3-ff937188aec6> (6))]), location: <ipython-input-3-ff937188aec6> (6)
[1m
File "<ipython-input-3-ff937188aec6>", line 6:[0m
[1mdef test():
[1m    return [1, [2]]
[0m    [1m^[0m[0m
[0m
This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the 

# Estimation

In [96]:
import SimulatedMinimumDistance as SMD

Moment and weight functions

In [124]:
def mom_fun(data):
    mask_mh = (data.states==6) | (data.states==8)
    mask_ml = (data.states==5) | (data.states==7)
    mask_fh = (data.states==1) | (data.states==3)
    mask_fl = (data.states==0) | (data.states==2)
    return np.hstack((np.nanmean(data.probs[:,mask_mh],axis=1),
                      np.nanmean(data.probs[:,mask_ml],axis=1),
                      np.nanmean(data.probs[:,mask_fh],axis=1),
                      np.nanmean(data.probs[:,mask_fl],axis=1)
                    ))*100

def weight_fun(data):
    mask_mh = (data.states==6) | (data.states==8)
    mask_ml = (data.states==5) | (data.states==7)
    mask_fh = (data.states==1) | (data.states==3)
    mask_fl = (data.states==0) | (data.states==2)
    std = np.hstack((np.nanstd(data.probs[:,mask_mh]*100,axis=1),
                     np.nanstd(data.probs[:,mask_ml]*100,axis=1),
                     np.nanstd(data.probs[:,mask_fh]*100,axis=1),
                     np.nanstd(data.probs[:,mask_fl]*100,axis=1)
                    ))
    return np.eye(len(std))*std

Create simulated data

In [123]:
data = SingleClass(Na=190, poc=10, a_max=10, simN=40000, simT=20)
data.par.simStates = np.array(list(range(8))*5000)
data.solve()
data.simulate()

In [125]:
mom_data = mom_fun(data.sim)
weight = weight_fun(data.sim)

Preparing for estimation

In [126]:
true = [data.par.alpha_0_female, data.par.alpha_0_male, data.par.alpha_1]
theta0 = [i*1.5 for i in true]
add_str = '_est'
est_par = ("alpha_0_female", "alpha_0_male", "alpha_1") # remember to be list if only 1 var

In [127]:
model_base = SingleClass(Na=190, poc=10, a_max=10, simN=40000, simT=20)
model_base.par.simStates = np.array(list(range(8))*5000)
model_base.par.sim_seed = 2019
model_base._simulate_prep()

In [128]:
# Different seeeds
print('data seed:',data.par.sim_seed)
print('estimation seed',model_base.par.sim_seed)

data seed: 1998
estimation seed 2019


In [129]:
# And different draws
print(np.allclose(data.sim.unif,model_base.sim.unif),
      np.allclose(data.sim.deadP,model_base.sim.deadP),
      np.allclose(data.sim.inc_shock,model_base.sim.inc_shock))

False False False


# Visualize objective function

In [156]:
n_lst = [1000,2000,3000,4000]
y_lst = []

for n in n_lst:
    model_base.par.simN = 8*n
    model_base.par.simStates = np.array(list(range(8))*n)
    model_base.par.simM_init = 5*np.ones(model_base.par.simN)
    model_base.simulate()
    mom_sim = mom_fun(model_base.sim)
    diff = mom_data - mom_sim
    obj = (np.transpose(diff) @ weight) @ diff
    y_lst.append(obj)

ZeroDivisionError: division by zero

In [ ]:
plt.plot(n_lst,y_lst)

Estimate

In [130]:
smd_base = SMD.SimulatedMinimumDistance(model_base,mom_data,mom_fun,print_iter=True,options={'disp':True})
smd_base.est_par = est_par
smd_base.estimate(theta0,weight)
theta_base = smd_base.est
theta_base-true

 alpha_0_female=0.253 alpha_0_male=0.240 alpha_1=0.080 -> 3615.1474
 alpha_0_female=0.266 alpha_0_male=0.240 alpha_1=0.080 -> 4072.0490
 alpha_0_female=0.253 alpha_0_male=0.252 alpha_1=0.080 -> 4059.5013
 alpha_0_female=0.253 alpha_0_male=0.240 alpha_1=0.083 -> 3759.7614
 alpha_0_female=0.241 alpha_0_male=0.248 alpha_1=0.082 -> 3588.9179
 alpha_0_female=0.228 alpha_0_male=0.252 alpha_1=0.083 -> 3460.2423
 alpha_0_female=0.237 alpha_0_male=0.236 alpha_1=0.085 -> 3130.9047
 alpha_0_female=0.228 alpha_0_male=0.228 alpha_1=0.087 -> 2717.9516
 alpha_0_female=0.220 alpha_0_male=0.240 alpha_1=0.083 -> 2781.7344
 alpha_0_female=0.197 alpha_0_male=0.240 alpha_1=0.090 -> 2507.0991
 alpha_0_female=0.169 alpha_0_male=0.240 alpha_1=0.095 -> 2300.4547
 alpha_0_female=0.183 alpha_0_male=0.220 alpha_1=0.094 -> 1723.4121
 alpha_0_female=0.161 alpha_0_male=0.204 alpha_1=0.099 -> 1151.1759
 alpha_0_female=0.152 alpha_0_male=0.208 alpha_1=0.105 -> 1338.9430
 alpha_0_female=0.093 alpha_0_male=0.207 alpha_1

KeyboardInterrupt: 